# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
import findspark
findspark.init('/home/czh/spark-2.4.6-bin-hadoop2.7/')

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.sql.functions import corr
from pyspark.ml.regression import LinearRegression

In [3]:
spark = SparkSession.builder.appName('project').getOrCreate()

In [4]:
# Read in the data
df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [5]:
df.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)

In [6]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [7]:
# Check if the cruise line is related to the crew
df.select('Cruise_line', 'crew').groupBy('Cruise_line').mean().show()

+-----------------+------------------+
|      Cruise_line|         avg(crew)|
+-----------------+------------------+
|            Costa|               7.7|
|              P&O| 8.588333333333333|
|           Cunard|10.246666666666668|
|Regent_Seven_Seas|             3.146|
|              MSC|              7.54|
|         Carnival| 10.13590909090909|
|          Crystal|             5.905|
|           Orient|               3.5|
|         Princess|  9.32235294117647|
|        Silversea|            2.4725|
|         Seabourn|1.6000000000000003|
| Holland_American| 6.215000000000002|
|         Windstar|1.1866666666666668|
|           Disney|              9.45|
|        Norwegian| 8.087692307692308|
|          Oceania|               4.0|
|          Azamara|              3.55|
|        Celebrity|             8.072|
|             Star| 5.363333333333333|
|  Royal_Caribbean|10.016521739130434|
+-----------------+------------------+



In [8]:
# Transform the cruise line into idx (numbers)
idxer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_idx')
idx = idxer.fit(df).transform(df)

In [9]:
idx.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_idx: double (nullable = false)



In [10]:
idx.select('Cruise_line_idx').show()

+---------------+
|Cruise_line_idx|
+---------------+
|           16.0|
|           16.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
|            1.0|
+---------------+
only showing top 20 rows



In [11]:
idx.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_idx']

In [12]:
idx.select(corr('crew', 'Cruise_line_idx')).show()

+---------------------------+
|corr(crew, Cruise_line_idx)|
+---------------------------+
|       -0.48332562728617057|
+---------------------------+



In [13]:
# Generate the dataframe for ML
assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'length', 'cabins',
                                       'passenger_density', 'Cruise_line_idx'], 
                            outputCol = 'features')

output = assembler.transform(idx)
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[26.0,47.262,14.8...|
|[11.0,110.0,29.74...|
|[17.0,101.353,26....|
|[22.0,70.367,20.5...|
|[15.0,70.367,20.5...|
|[23.0,70.367,20.5...|
|[19.0,70.367,20.5...|
|[6.0,110.23899999...|
|[10.0,110.0,29.74...|
|[28.0,46.052,14.5...|
|[18.0,70.367,20.5...|
|[17.0,70.367,20.5...|
|[11.0,86.0,21.24,...|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,9...|
|[15.0,70.367,20.5...|
|[12.0,88.5,21.24,...|
|[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows



In [14]:
# Split the data into train and test sets
data = output.select('crew', 'features')
train, test = data.randomSplit([0.7, 0.3])

In [15]:
# Create the instance of the model
lr = LinearRegression(labelCol='crew')

In [16]:
# Fit on the train set
lrModel = lr.fit(train)

In [17]:
lrModel.coefficients

DenseVector([-0.0131, 0.0062, -0.1382, 0.5276, 0.795, -0.0075, 0.0366])

In [18]:
lrModel.summary.r2

0.9490425787723434

In [19]:
# Evaluate on the test set
test_results = lrModel.evaluate(test)

In [20]:
test_results.r2

0.8692601075550593

In [21]:
# Predict on the test set
unlabeled_test = test.select('features')
lrModel.transform(unlabeled_test).show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[6.0,30.276999999...|4.2323567406431675|
|[14.0,30.27699999...|3.5229988437105164|
|[23.0,25.0,7.76,6...|  3.93481714594463|
|[15.0,30.27699999...| 3.866798067149237|
|[21.0,38.0,10.56,...|4.4278758558012274|
|[10.0,46.0,7.0,6....| 2.901309085064098|
|[25.0,38.0,7.49,6...| 4.167536523375334|
|[29.0,45.0,11.78,...|  5.36652643996916|
|[14.0,63.0,14.4,7...| 6.681999699919217|
|[19.0,55.451,12.6...| 5.812610286875859|
|[20.0,55.451,12.6...|5.7995455185703175|
|[13.0,61.0,13.8,7...| 6.523714511316095|
|[27.0,53.872,14.9...| 6.764575722588509|
|[25.0,42.0,15.04,...| 6.366339847192291|
|[27.0,53.872,14.9...| 6.996701258769175|
|[15.0,78.491,24.3...| 8.320162380783513|
|[21.0,47.225,13.6...|6.0107836687050185|
|[23.0,48.563,20.2...| 5.899846617080534|
|[24.0,40.05300000...| 6.299772829758519|
|[16.0,78.491,24.3...| 8.307097612477971|
+--------------------+------------